### Описание реализованного функционала ноутбука
1. Построение графа связанных бизнес-процессов (БП) в зависимости параметров метода построения
2. Построение короткого пути между двумя выбранными БП в зависимости параметров метода построения
3. Построение графа соседей для выбранной вершины в зависимости от максимального радиуса  параметров метода построения

In [ ]:
import pandas as pd
import os
import re
import itertools
import pymorphy2
import networkx as nx
import pandas as pd
%matplotlib inline
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
pd.set_option('display.max_columns', None)  # вывод всех столбцов фрайма
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from itertools import accumulate
import datetime
import preprocessing
import sklearn
from pyvis import network as net

In [ ]:
DATA_PATH = '/home/jovyan/work/data'
PATH_TO_EUCLIDEAN_DIST=f'{DATA_PATH}/Matrix_For_Graphs_v2/dist_matrix_euc_all_codes.csv'
PATH_TO_COSINE_DISTANCE=f'{DATA_PATH}/Matrix_For_Graphs_v2/dist_matrix_cos_all_codes.csv'
PATH_TO_LOAD_TABLE=f'{DATA_PATH}/Matrix_For_Graphs_v2/load_table.csv'
ADJACENCY_MATRIX_NEAREST_NEIGHBORS=f'{DATA_PATH}/Matrix_For_Graphs_v2/dist_df_euc.csv'

### Вспомогательные функции

In [ ]:
def get_data_for_graph(type_bulding_graph):
    if type_bulding_graph=="euclidean":
        data=pd.read_csv(PATH_TO_EUCLIDEAN_DIST,index_col="business_processes")
        
    elif type_bulding_graph=="cosine":
        data=pd.read_csv(PATH_TO_COSINE_DISTANCE,index_col="business_processes")
        
    elif type_bulding_graph=="neigbors":
        data=pd.read_csv(ADJACENCY_MATRIX_NEAREST_NEIGHBORS,sep=';')
        
    else: 
        return print("Получено некорректное значение переменной type_bulding_graph")
        
    return data

def сreate_edges_for_graph(treshold_dist,data):
    
    Edges_list=[(index,сolumn,data[index][сolumn]) for index,сolumn in itertools.permutations(data.index.to_list(),2)
               if (data[index][сolumn]<=treshold_dist)&(data[index][сolumn]!=0)]
    
    Edges=pd.DataFrame(Edges_list)
    
    Edges.columns=["source",
                   'target',
                   'weight']

    
    max_weight=Edges.weight.max()
    min_weight=Edges.weight.min()
    
    def normalise_weight(weight,max_weight=max_weight,min_weight=min_weight):
        return int((1-(weight-min_weight)/(max_weight-min_weight))*100)
    
    Edges.weight=Edges.weight.map(normalise_weight)
    return Edges

def сreate_edges_for_neighbors_graph(data,treshold_dist,number_neighbors):
    
    data=data[data.dist<=treshold_dist]
    data_for_edges=[]
    
    for i in range(number_neighbors):
        edges_min_dist=[(row['id1'],row['id2'],row['dist'],i) for i,row in data.iterrows() if data[
            data.id1==row['id1']]['dist'].min()==row['dist']]
        
        index_edges_min_dist=[el[-1] for el in edges_min_dist]
        data.drop(index_edges_min_dist,axis=0,inplace=True)
        data_for_edges=data_for_edges+edges_min_dist

    edges=pd.DataFrame(data_for_edges,columns=["source",
                                               'target',
                                               'weight',
                                               'index'])
    return edges
    
def create_graph(edges):
    G = nx.from_pandas_edgelist(df=edges,
                                source="source",
                                target='target',
                                edge_attr='weight')
    
    G.add_nodes_from(nodes_for_adding=edges.source.tolist()+edges.target.tolist())
    
    links_attributes = {tuple(row[["source", 'target']]): {
        'weight': row['weight']} for i,row in edges.iterrows()}

    G.add_edges_from(links_attributes) 
    nx.set_edge_attributes(G=G, values=links_attributes) 
    return G

def shortest_path_edges(graph,node1,node2):
    if (node1 in graph)&(node2 in graph):
        if nx.has_path(graph,source=node1, target=node2):
            path=nx.shortest_path(graph, source=node1, target=node2)

            path_edges = pd.DataFrame(zip(path,path[1:]),columns=["source",
                                                                  'target'])
            path_edges['weight']=1
            return path_edges
        else:
            return "Между данными БП не существует связи при данных параметрах определения связей. Выберите другие вершины или измените параметры построения графа."
    else:
        return "Один БП отстутствует в графе. Выберите другие БП или измените параметры построения графа."



color_functionality={'01.01':'yellow','01.02':'red','01.04':'chartreuse','01.05':'mediumorchid','01.07':'blue'}
chain1=set(['01.01.01.01','01.01.02','01.01.03',
                        '01.01.01.02.01',
                        '01.01.01.02.02',
                        '01.01.03.02',
                        '01.01.03.03.02',
                        '01.01.03.03.01',
                        '01.01.03.04',
                        '01.01.03.05'])
chain2=['13.03.02','13.03.12.01','13.06.07.04','13.07.11','13.03.07.02','13.03.10.01']
chain3=['01.02.01.02.03','01.02.01.02.01','01.02.01','01.02.01.06','11.05','11.05.01']
chain4=['14.06.01','14.01.01','14.00','14','14.06.02','14.03.05','14.06.04','14.02.01','14.02.02']
#Цепочки для демонстрации работы модуля

load_table=pd.read_csv(PATH_TO_LOAD_TABLE,index_col='code')
#Таблица нагрузки на звенья цепочек
def colorize_node(node,type_colorize):
    code=re.sub(" +", " ", node).split(" ")[0]
    color_node='black'
    color_size=25
    label=code
    title=node
    if type_colorize=='functionality':
        if code[0:5] in color_functionality:
            color_node=color_functionality[code[0:5]]
            color_size=50
    if type_colorize=='chains':
        if code in chain1:
            color_node='orange'
            color_size=50
            label=node
        if code in chain2:
            color_node='green'
            color_size=50
            label=node
        if code in chain3:
            color_node='yellow'
            color_size=50
            label=node
        if code in chain4:
            color_node='blue'
            color_size=50
            label=node
    if type_colorize=='load':
        if code in load_table.index:
            code_inform=load_table.loc[code]
            
            color_size=int(code_inform['counts'])
            
            if color_size>50:
                color_node='red'
            elif color_size<10:
                color_node='red'
            if color_size<2:
                color_size=5
                color_node='blue'
            title=code_inform['describtion']
    return code,color_node,color_size,label,title 

def colorize_edge(node1,node2,type_colorize):
    color_edge='silver'
    color_node1= colorize_node(node1,type_colorize)[1]
    color_node2= colorize_node(node2,type_colorize)[1]
    if (color_node1==color_node2)&(color_node1!='black'):
        color_edge=color_node1
    return color_edge
        
    
def drow_graph(Edges,type_colorize):
    options = {
        'shape' : 'dot',
        'size' : 15,
        'color' : '#ECBF26', 

        'font' : {
            'size' : 16,
            'color' : '#ffffff'
        },
        'borderWidth' : 2
    }
    
    got_net = net.Network(height="900px",
                          width="120%",
                          bgcolor="white",
                          font_color="black",
                          notebook=True)

    
    got_net.barnes_hut(gravity=-10000) #-10000
    got_data = Edges
    
    sources = got_data['source']
    targets = got_data['target']
    weights = got_data['weight']

    edge_data = zip(sources, targets, weights)

    for e in edge_data:
        src = e[0]
        dst = e[1]
        w = e[2]

        code1,color_node,color_size,label,title=colorize_node(src,type_colorize) 
        got_net.add_node(n_id=code1, label=label,title=src+"\n"+title,color=color_node,size=color_size,borderWidth=2)
        
        code2,color_node,color_size,label,title=colorize_node(dst,type_colorize) 
        got_net.add_node(n_id=code2, label=label,title=dst+"\n"+title,color=color_node,size=color_size,borderWidth=2)
        
        color_edge=colorize_edge(src,dst,type_colorize)
        got_net.add_edge(code1, code2, value=w,color=color_edge,font={'size':40})

    neighbor_map = got_net.get_adj_list()
    
    for node in got_net.nodes:
        node["title"] = node["title"].split('\n')[0]+" Employees:<br>" + "<br>".join(node["title"].split('\n')[1:])


    
    
#     for node in got_net.nodes:
#         node["title"] += '\n'+" Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
#         node["value"] = len(neighbor_map[node["id"]])
    
    return got_net

def drow_graph_nx(graph):
    
    g=net.Network(height="750px",
                  width="100%",
                  bgcolor="#222222",
                  font_color="aqua",
                  notebook=True)
    
    g.from_nx(graph)
    g.barnes_hut(gravity=-5000)
    return g

{'shape' : 'dot',
 'size' : 15,
'color' : '#ECBF26',
'font' : {'size' : 16,'color' : '#ffffff'}}


def drow_eho_graph(G,node1,radius):
    if node1 in G:
        g=net.Network(height="750px",
                      width="100%",
                      bgcolor="#222222",
                      font_color="aqua",
                      notebook=True)

        G=nx.ego_graph(G=G,
                       n=node1,
                       radius=radius)
        
        g.from_nx(G)
        g.barnes_hut(gravity=-10000)
        return g
    else:
        return "Данная вершина не существует. Выберите другую вершину или измените параметры построения графа."

def display_graph(g, filename='_graph_tmp.html'):
    return g.write_html(filename, notebook=True)

def get_all_nodes_list():
    data=pd.read_csv(PATH_TO_EUCLIDEAN_DIST,usecols=["business_processes"])
    return data.business_processes.to_list()

### 3 функции, реализующие функционал

In [ ]:
def all_nodes_graph(treshold_dist,
                    type_bulding_graph,
                    number_neighbors=1,
                    type_colorize='functionality'):
    
    data=get_data_for_graph(type_bulding_graph)
    
    if type_bulding_graph in ("euclidean","cosine"):
        
        all_edges=сreate_edges_for_graph(treshold_dist,
                                         data)
        
    elif type_bulding_graph=="neigbors":
        
        all_edges=сreate_edges_for_neighbors_graph(data,
                                                   treshold_dist,
                                                   number_neighbors)
    
    return drow_graph(all_edges,type_colorize)

In [ ]:
def eho_graph(node1,
              radius,
              treshold_dist,
              type_bulding_graph,
              number_neighbors=1):
    
    data=get_data_for_graph(type_bulding_graph)
    
    if type_bulding_graph in ("euclidean","cosine"):
        all_edges=сreate_edges_for_graph(treshold_dist,
                                         data)
    elif type_bulding_graph=="neigbors":
        all_edges=сreate_edges_for_neighbors_graph(data,
                                                   treshold_dist,
                                                   number_neighbors)

    
    graph=create_graph(all_edges)
    
    return drow_eho_graph(graph,node1,radius)

In [ ]:
def shortest_chain(treshold_dist,
                   type_bulding_graph,
                   node1='14.02.02',
                   node2='01.02.05.01',
                   number_neighbors=1,
                   type_graph='eho_graph'):
    
    data=get_data_for_graph(type_bulding_graph)
    
    if type_bulding_graph in ("euclidean","cosine"):
        
        all_edges=сreate_edges_for_graph(treshold_dist,
                                         data)
    elif type_bulding_graph=="neigbors":
        
        all_edges=сreate_edges_for_neighbors_graph(data,
                                                   treshold_dist,
                                                   number_neighbors)
        
    graph=create_graph(all_edges)
    edges=shortest_path_edges(graph,node1,node2)
    if type(edges)==str:
        return edges
    else:
        shortest_path_graph=create_graph(edges)
        return drow_graph_nx(shortest_path_graph)

### Иллюстрация работы функций

In [ ]:
ehog = eho_graph(treshold_dist=0.75,
                 type_bulding_graph='cosine',
                 number_neighbors=1,
                 node1='01.01.03.01 Управление программой ГРР',
                 radius=1) 
display_graph(ehog, '_echo_graph_result.html')

In [ ]:
shch = shortest_chain(treshold_dist=0.75,
                      type_bulding_graph='cosine',
                      node1='01.01.03.01 Управление программой ГРР',
                      node2='01.04.05 Бурение скважин')

display_graph(shch, '_shortest_chain_result.html')

In [ ]:
ang = all_nodes_graph(treshold_dist=0.75,
                      type_bulding_graph='cosine',
                      type_colorize='load')

display_graph(ang, '_all_nodes_graph_result.html')

### Сохранение результатов

In [ ]:
#iframe_template= '<iframe width="{width}" height="{height}" {options} srcdoc="{html}"></iframe>'

#def wrap_html_to_iframe(html, width, height, options=None):
    # escape quotes
#    prepared_html = html.replace('"', '&quot;')
#    iframe = iframe_template.format(width=width, height=height, options=options, html=prepared_html)
#    return iframe

#from datamall import dsl

#def save_to_results(graph):
    # method renders html representation, saves it to `html` property and writes to the file 
#     graph.write_html('_graph_tmp.html')
#     iframe = wrap_html_to_iframe(graph.html, width='100%', height='750px')
#     result = dsl.save_scenario_results(iframe)
#     return result

In [ ]:
## save eho graph
#save_to_results(ehog)

## all nodes graph
#save_to_results(ang)

## save shortest chain graph
#save_to_results(shch)

### Основная функция

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [ ]:
current_graph = None

# save_button = widgets.Button(
#     description='Save graph',
#     disabled=False,
#     button_style='success',
#     tooltip='Click to save currently displayed graph',
#     icon='save'
# )

# def on_save_click(b):
#     save_to_results(current_graph)
#     save_button.disabled=True
#     save_button.icon="check"

# save_button.on_click(on_save_click)

# def activate_save_button():
#     save_button.disabled=False
#     save_button.icon="save"

@interact
def show_graph(type_bulding_graph = ["cosine",'euclidean'],
               type_graph=["All","Neighbors","Shortest_path"],
               treshold_dist=(0.65,0.75,0.02),
               number_neighbors=(1,4,1),
               node1=get_all_nodes_list(),
               node2=get_all_nodes_list(),
               type_colorize=['functionality','chains','load'], 
               radius=(1,4,1)):
        
#         save_button.disabled = True
    
        global current_graph

        
        print('Процесс построения графа {} запущен.'.format(type_graph))
        
        if type_graph=="All":
            current_graph=all_nodes_graph(
                treshold_dist=treshold_dist,
                type_bulding_graph=type_bulding_graph,
                type_colorize=type_colorize
            )
            
        
        elif type_graph=="Neighbors":
            current_graph=eho_graph(
                node1=node1,
                radius=radius,
                treshold_dist=treshold_dist,
                type_bulding_graph=type_bulding_graph,
            )
            
        
        elif type_graph=="Shortest_path":
            current_graph=shortest_chain(
                treshold_dist=treshold_dist,
                type_bulding_graph=type_bulding_graph,
                node1=node1,
                node2=node2,
                number_neighbors=number_neighbors
            )
            
        if isinstance(current_graph, net.Network):
            print('Граф {} построен.'.format(type_graph))
#             activate_save_button()
            return display_graph(current_graph)
        else:
            return current_graph

In [ ]:
#display(save_button)

Нюанс - графы euclidean и neigbors строятся относительно долго при высоких значениях treshold_dist

### Параметры функции
###### type_bulding_graph
Mетод построения графов бизнес-процессов. Значения:
1. euclidean - ребро проводится, если евклидово расстояние между средним текстовым описанием двух БД меньше порога(treshold_weight)
2. cosine - ребро проводится, если косинусное расстояние между средним текстовым описанием двух БД меньше порога(treshold_weight)
3. neigbors - ребро между двумя БП проводится если один из них входит в k(number_neighbors)-ближайших соседей другого в терминах евклидова расстояния (в общем графе параметра пока нет)

###### type_graph
Тип графа для визуализации. Значения:
1. All - визуализация всех вершин графа. 
2. Neighbors - визуализация соседей для выбранной вершины (node1) с выбранным радиусом (radius)
3. Shortest_path - визуализация короткого пути между двумя выбранными вершинами (node1 и node2)

###### treshold_dist
Пороговое значение для расстояния между вершинами. Если расстояние меньше порога - ребро проводится.

###### type_colorize
Тип раскраски графа свящи всех бизнес процессов. Значения:
1. functionality- раскраска по функциональным отделениям
2. chains- раскраска по функциональным отделениям

###### number_neighbors
Количество соседей для вершины метода neigbors (type_bulding_graph)

###### node1
Вершина для построения графа соседей и короткого пути
###### node2
Вторая вершина для построения графа короткого пути
###### radius
Радиус для построения графа соседей